In [1]:
!echo $SHELL
!echo $PATH
!echo $JOURNAL_ROOT

/bin/zsh
/Users/tobiaspucher/miniconda3/bin:/opt/homebrew/opt/node@22/bin:/Users/tobiaspucher/miniconda3/bin:/Users/tobiaspucher/miniconda3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Library/Apple/usr/bin:/Library/TeX/texbin:/Users/tobiaspucher/.cargo/bin:/Users/tobiaspucher/.local/bin:/Users/tobiaspucher/Library/Application Support/JetBrains/Toolbox/scripts
/Users/tobiaspucher/Library/CloudStorage/OneDrive-Personal/2025/trade_journal


In [2]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.ERROR)
import random
random.seed(0)
import os
os.getenv('JOURNAL_ROOT')

'/Users/tobiaspucher/Library/CloudStorage/OneDrive-Personal/2025/trade_journal'

In [3]:
from src.tradecli import *
from src.utils import *
from src.journal import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [6]:
config = Config(
    include_patterns=[r'^type_1_', r'^type_3_', r'^risk_reward_ratio$', r'^side'],  
    exclude_patterns=[r'^management_strategy$'],         
    #force_values={'side': 'short', 'side': 'long'},
)

full_df = j.to_dataframe(config)
#full_df = full_df[full_df['side'] == 'long']
display(full_df)

,type_1_h4,type_1_w,type_3_m15,type_1_M,type_3_m30,risk_reward_ratio,type_1_m30,side,type_3_m1,type_3_m5,type_3_M,type_1_m1,type_3_h1,type_3_d,type_3_h4,type_3_w,type_1_m15,type_1_h1,type_1_m5,type_1_d
0,False,False,True,False,False,2.000000,False,long,False,False,False,True,False,False,False,False,False,False,False,False
1,False,False,False,False,False,0.666667,False,long,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,10.000000,False,long,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,10.000000,False,long,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,3.000000,False,long,False,False,False,False,False,False,False,False,False,False,False,True
5,False,False,False,True,False,3.943182,False,long,False,False,False,False,False,False,False,False,False,False,False,False
6,True,False,False,False,False,3.323529,False,long,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,7.927273,False,long,False,False,False,True,False,False,False,False,False,False,False,False
8,False,False,False,False,False,8.811321,True,long,False,False,False,False,False,False,False,False,False,False,False,False
9,True,False,False,False,False,2.986486,False,long,False,False,False,False,False,False,False,False,False,False,False,False


In [7]:
!rm -rf output
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
j.to_markdown(output_dir='output',)

2025-01-23 16:03:21 - root - INFO - Journal root: /Users/tobiaspucher/Library/CloudStorage/OneDrive-Personal/2025/trade_journal
2025-01-23 16:03:21 - root - INFO - CMD Journal root: /Users/tobiaspucher/Library/CloudStorage/OneDrive-Personal/2025/trade_journal
2025-01-23 16:03:21 - root - INFO - Assets root: /Users/tobiaspucher/Library/CloudStorage/OneDrive-Personal/2025/trade_journal/assets
2025-01-23 16:03:21 - root - INFO - Export root: /Users/tobiaspucher/Library/CloudStorage/OneDrive-Personal/2025/trade_journal/export
2025-01-23 16:03:21 - root - INFO - Journal root: /Users/tobiaspucher/Library/CloudStorage/OneDrive-Personal/2025/trade_journal
2025-01-23 16:03:21 - root - INFO - CMD Journal root: /Users/tobiaspucher/Library/CloudStorage/OneDrive-Personal/2025/trade_journal
2025-01-23 16:03:21 - root - INFO - Assets root: /Users/tobiaspucher/Library/CloudStorage/OneDrive-Personal/2025/trade_journal/assets
2025-01-23 16:03:21 - root - INFO - Export root: /Users/tobiaspucher/Library/C

In [8]:
df = j.to_dataframe(config=None)
df

,trade_uid,entry_time,potential_risk_reward,outcome,tp_price,unit_test,type_2_M,type_1_h4,type_1_w,session,initial_return,type_3_m15,type_2_m30,type_1_M,type_3_m30,management_strategy,risk_reward_ratio,type_2_m1,type_1_m30,numerical_confidence,close_price,type_2_m5,side,SL_distance,type_3_m1,type_2_w,type_3_m5,return,htf_poi_ltf_confirmation,type_3_M,potential_return,type_1_m1,type_2_d,type_3_h1,type_3_d,potential_price,account_name,type_3_h4,type_3_w,TP_distance,initial_risk_reward,type_1_m15,type_2_h4,type_2_m15,type_1_h1,type_2_h1,account,entry_price,sl_price,type_1_m5,type_1_d,htf_poi_ltf_confirmation_False,htf_poi_ltf_confirmation_True,management_strategy_be_after_1r,management_strategy_be_after_push,management_strategy_no_management,management_strategy_strategy_2,outcome_loss,outcome_win
0,1001,2025-01-23 16:02:04.728513,NaN,NaN,1.1100,False,False,False,False,new_york,0.0100,True,False,False,False,no_management,2.000000,False,False,3.0,NaN,False,long,0.0050,False,False,False,NaN,False,False,NaN,True,False,False,False,NaN,default,False,False,0.0100,2.000000,False,False,False,False,False,None,1.1000,1.0950,False,False,True,False,False,False,True,False,False,False
1,1002,2025-01-23 16:02:04.728553,NaN,loss,1.2100,False,False,False,False,new_york,0.0100,False,False,False,False,no_management,0.666667,False,False,2.0,1.1855,False,long,0.0150,False,False,False,-0.0145,True,False,NaN,False,False,False,False,NaN,default,False,False,0.0100,0.666667,False,False,False,False,True,None,1.2000,1.1850,False,False,False,True,False,False,True,False,True,False
2,1003,2024-12-27 10:00:01.000000,NaN,NaN,1.3300,False,False,False,False,london,0.0300,False,False,False,False,no_management,10.000000,False,False,NaN,NaN,False,long,0.5000,False,False,False,NaN,False,False,NaN,False,False,False,False,NaN,default,False,False,0.0300,10.000000,False,False,False,False,False,None,1.3000,1.2970,False,False,True,False,False,False,True,False,False,False
3,1004,2024-12-25 10:00:00.000000,5.000000,win,1.3300,False,False,False,False,london,0.0300,False,False,False,False,strategy_2,10.000000,False,False,4.0,1.3050,False,long,0.5000,False,False,False,0.0050,False,False,0.0150,False,False,False,False,1.3150,default,False,False,0.0300,10.000000,False,False,False,False,False,None,1.3000,1.2970,False,False,True,False,False,False,False,True,False,True
4,1005,2025-01-23 16:02:04.729619,3.397959,win,1.1410,True,False,False,False,new_york,0.0294,False,False,False,False,be_after_1r,3.000000,False,False,3.0,1.1248,False,long,0.0098,False,False,False,0.0132,False,False,0.0333,False,True,False,False,1.1449,None,False,False,0.0294,3.000000,False,False,False,False,False,test_account,1.1116,1.1018,False,True,True,False,True,False,False,False,False,True
5,1006,2025-01-23 16:02:04.729668,1.590909,NaN,1.1560,True,False,False,False,new_york,0.0347,False,False,True,False,no_management,3.943182,False,False,3.0,NaN,False,long,0.0088,False,False,False,NaN,True,False,0.0140,False,False,False,False,1.1353,None,False,False,0.0347,3.943182,False,False,True,False,False,test_account,1.1213,1.1125,False,False,False,True,False,False,True,False,False,False
6,1007,2025-01-23 16:02:04.729709,6.558824,loss,1.1550,True,False,True,False,new_york,0.0226,False,False,False,False,be_after_push,3.323529,False,False,4.0,1.1206,False,long,0.0068,False,True,False,-0.0118,False,False,0.0446,False,False,False,False,1.1770,None,False,False,0.0226,3.323529,False,False,False,False,False,test_account,1.1324,1.1256,False,False,True,False,False,True,False,False,True,False
7,1008,2025-01-23 16:02:04.729745,5.400000,win,1.1902,True,True,False,False,new_york,0.0436,False,False,False,False,be_after_push,7.927273,False,False,5.0,1.1580,False,long,0.0055,False,False,False,0.0114,True,False,0.0297,True,False,False,False,1.1763,None,False,False,0.0436,7.927273,False,False,False,False,False,test_account,1.1466,1.1411,False,False,False,True,False,True,False,False,False,True
8,1009,2025-01-23 16:02:04.729778,5.264151,NaN,1.1

In [12]:
df_test = df.copy()
features = ['type_1_m5', 'type_2_m5', 'type_3_m5', 'htf_poi_ltf_confirmation', 'return']
df_test = df_test[features]
df_test = df_test.dropna()
exptected_return = df_test['return'].mean()
print('Expected return full search space:', exptected_return)
if exptected_return > 0:
    print('Expected return is positive')
else:
    print('Expected return is negative')
df_test

display(df_test.dtypes)
display(df_test.describe())
display(df_test)


report_best_performing_tags_value(df_test, y_col='return', score_mode='mean', method='brute_force')

Expected return full search space: 0.0031624999999999848
Expected return is positive


type_1_m5                       bool
type_2_m5                       bool
type_3_m5                       bool
htf_poi_ltf_confirmation    category
return                       float64
dtype: object

,return
count,8.000000
mean,0.003162
std,0.012738
min,-0.014500
25%,-0.009250
50%,0.008200
75%,0.013325
max,0.016700


,type_1_m5,type_2_m5,type_3_m5,htf_poi_ltf_confirmation,return
1,False,False,False,True,-0.0145
3,False,False,False,False,0.0050
4,False,False,False,False,0.0132
6,False,False,False,False,-0.0118
7,False,False,False,True,0.0114
10,False,False,False,True,-0.0084
11,False,True,False,False,0.0137
12,False,False,False,False,0.0167


Best subset: ('type_1_m5',)
Best combination: (False,)
Best score: 0.0031624999999999848
Best subset: ('type_2_m5',)
Best combination: (True,)
Best score: 0.013700000000000045
Best subset: ('type_2_m5',)
Best combination: (True,)
Best score: 0.013700000000000045


In [13]:
import src.tradecli
cfg = src.tradecli.ReportOptunaConfig(study_name='default_1')
res = report_best_performing_tags_value(df_test, 
                                  y_col='return', 
                                  score_mode='mean', method='optuna', config=cfg)

[I 2025-01-23 16:05:07,771] A new study created in memory with name: default_1


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-01-23 16:05:07,781] Trial 0 finished with value: -0.0038333333333333552 and parameters: {'include_type_1_m5': True, 'value_type_1_m5': False, 'include_type_2_m5': False, 'include_type_3_m5': True, 'value_type_3_m5': False, 'include_htf_poi_ltf_confirmation': True, 'value_htf_poi_ltf_confirmation': True}. Best is trial 0 with value: -0.0038333333333333552.
[I 2025-01-23 16:05:07,783] Trial 1 finished with value: 0.0031624999999999848 and parameters: {'include_type_1_m5': False, 'include_type_2_m5': False, 'include_type_3_m5': True, 'value_type_3_m5': False, 'include_htf_poi_ltf_confirmation': False}. Best is trial 1 with value: 0.0031624999999999848.
[I 2025-01-23 16:05:07,786] Trial 2 finished with value: 0.013700000000000045 and parameters: {'include_type_1_m5': False, 'include_type_2_m5': True, 'value_type_2_m5': True, 'include_type_3_m5': False, 'include_htf_poi_ltf_confirmation': False}. Best is trial 2 with value: 0.013700000000000045.
[I 2025-01-23 16:05:07,791] Trial 3 f

In [14]:
res

(<optuna.study.study.Study at 0x1682b4dd0>,
 {'include_type_1_m5': False,
  'include_type_2_m5': True,
  'value_type_2_m5': True,
  'include_type_3_m5': False,
  'include_htf_poi_ltf_confirmation': False},
 0.013700000000000045)

In [15]:
res[0].best_trial
study = res[0]

In [16]:
# visualize stuff
import optuna
import plotly
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_slice
from optuna.visualization import plot_param_importances

# disable optuna warnings
optuna.logging.set_verbosity(optuna.logging.ERROR)

plot_optimization_history(study).show()
plot_param_importances(study).show()
plot_slice(study).show()

In [18]:
!rm -rf output/studies.md
import pprint
def markdown_report_study(study:optuna.Study, output_dir='output'):
    SECTION_OPTUNA_STUDIES = '## Optuna Studies'
    SECTION_BEST_TRIAL = f'### {study.study_name} - {study.best_trial.number}'
    GENERATION_TIME = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    best_trial = study.best_trial
    best_trial_params = best_trial.params
    best_trial_value = best_trial.value
    best_trial_number = best_trial.number
    best_trial_number
    best_trial_params
    best_trial_value
    # add the params as nicely formatted json inside code block
    best_trial_params_str = f'''
```json
{pprint.pformat(best_trial_params)}
```
''' 
    
    best_trial_str = f'''
{SECTION_BEST_TRIAL}
- generation time: {GENERATION_TIME}
- best trial value: {best_trial_value}
- best trial params: {best_trial_params_str}
'''
    with open(f'{output_dir}/studies.md', 'a') as f:
        f.write(best_trial_str)

    
    # find the section or create it and add md link to the best trial in the study
    with open(f'{output_dir}/index.md', 'r') as f:
        lines = f.readlines()
        found = False
        for i, line in enumerate(lines):
            if SECTION_OPTUNA_STUDIES in line:
                found = True
                break
        if not found:
            lines.append(f'\n{SECTION_OPTUNA_STUDIES}\n')
        lines.append(f'- [[studies.md{SECTION_BEST_TRIAL}]]\n')
    with open(f'{output_dir}/index.md', 'w') as f:
        f.write(''.join(lines))
        

markdown_report_study(study, output_dir='output')

In [21]:
df = j.to_dataframe()
print(df.columns)

Index(['trade_uid', 'entry_time', 'potential_risk_reward', 'outcome',
       'tp_price', 'unit_test', 'type_2_M', 'type_1_h4', 'type_1_w', 'session',
       'initial_return', 'type_3_m15', 'type_2_m30', 'type_1_M', 'type_3_m30',
       'management_strategy', 'risk_reward_ratio', 'type_2_m1', 'type_1_m30',
       'numerical_confidence', 'close_price', 'type_2_m5', 'side',
       'SL_distance', 'type_3_m1', 'type_2_w', 'type_3_m5', 'return',
       'htf_poi_ltf_confirmation', 'type_3_M', 'potential_return', 'type_1_m1',
       'type_2_d', 'type_3_h1', 'type_3_d', 'potential_price', 'account_name',
       'type_3_h4', 'type_3_w', 'TP_distance', 'initial_risk_reward',
       'type_1_m15', 'type_2_h4', 'type_2_m15', 'type_1_h1', 'type_2_h1',
       'account', 'entry_price', 'sl_price', 'type_1_m5', 'type_1_d',
       'htf_poi_ltf_confirmation_False', 'htf_poi_ltf_confirmation_True',
       'management_strategy_be_after_1r', 'management_strategy_be_after_push',
       'management_strategy_no

In [27]:
full_df = df.filter(regex='^type_|^htf_poi_ltf_confirmation$|^return$')
full_df.columns

Index(['type_2_M', 'type_1_h4', 'type_1_w', 'type_3_m15', 'type_2_m30',
       'type_1_M', 'type_3_m30', 'type_2_m1', 'type_1_m30', 'type_2_m5',
       'type_3_m1', 'type_2_w', 'type_3_m5', 'return',
       'htf_poi_ltf_confirmation', 'type_3_M', 'type_1_m1', 'type_2_d',
       'type_3_h1', 'type_3_d', 'type_3_h4', 'type_3_w', 'type_1_m15',
       'type_2_h4', 'type_2_m15', 'type_1_h1', 'type_2_h1', 'type_1_m5',
       'type_1_d'],
      dtype='object')

In [29]:
full_df['htf_poi_ltf_confirmation']

0     False
1      True
2     False
3     False
4     False
5      True
6     False
7      True
8      True
9      True
10     True
11    False
12    False
13     True
Name: htf_poi_ltf_confirmation, dtype: category
Categories (2, bool): [False, True]